In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wine-quality/winequalityN.csv


1)KNN-
0.9967365967365968 imputer:1 n_neighbours:3 p:1 weights=distance
{'p': 1, 'weights': 'distance'}
KNeighborsClassifier(n_neighbors=11, p=1, weights='distance')

2)Logistic Regression
0.9705878941263315
{'C': 10.0, 'penalty': 'l1'}
LogisticRegression(C=10.0, max_iter=1000000, penalty='l1', solver='saga')

3)Naive bayes Gaussian 0.9687645687645687 (without scaling) , .9748251748251748 (min max scaling )0.9902097902097902 (min-max+ power transformer)

4)Naive bayes multinomial 0.9221445221445221 (without scaling) 0.7575757575757576 (with sclaing )

5)Naive bayes bernoulli 0.7757575757575758(without scaling ) 0.7762237762237763 (with sclaing)

6)SVM 0.9967365967365968 {'C': 1.0, 'gamma': 1.0, 'kernel': 'rbf'}
SVC(gamma=1.0)

7)decision tree 0.9967365967365968

8) **bagging classifier svm** **0.9972027972027973**



In [5]:
df=pd.read_csv("/kaggle/input/wine-quality/winequalityN.csv")

In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB
None


In [ ]:
print(df.head(5))

In [9]:
X=(df.iloc[:,1:])
y=df["type"]

In [10]:
y=y.replace({"red":1,"white":0})

/tmp/ipykernel_95/3619773954.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y=y.replace({"red":1,"white":0})


In [11]:
print(y)

0       0
1       0
2       0
3       0
4       0
       ..
6492    1
6493    1
6494    1
6495    1
6496    1
Name: type, Length: 6497, dtype: int64


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# **K NEAREST NEIGHBOURS**

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
transformer = PowerTransformer()
scaler = MinMaxScaler()


weight_options = ['uniform', 'distance']
p=[1,2]
param_grid = dict(weights=weight_options,p=p)

accuracy=0;
a=0;
b=0;
for i in range(1,12):
    for j in range(1,12):
        imputer = KNNImputer(n_neighbors=j)
        X_train_imputed = imputer.fit_transform(X_train)
        X_test_imputed = imputer.transform(X_test)
        model = KNeighborsClassifier(n_neighbors=i)
        X_train_transformed = transformer.fit_transform(X_train_imputed)
        X_test_transformed = transformer.transform(X_test_imputed)
        scaled_X_train = scaler.fit_transform(X_train_transformed)
        scaled_X_test = scaler.transform(X_test_transformed)
        grid = GridSearchCV(model, param_grid, cv=10, scoring='accuracy', return_train_score=False)
        grid.fit(scaled_X_train, y_train)
        y_predicted = grid.predict(scaled_X_test)
        from sklearn.metrics import accuracy_score
        if(accuracy_score(y_test, y_predicted))>accuracy:
            accuracy=accuracy_score(y_test, y_predicted);
            a=i;
            b=j;
print(accuracy,a,b);
            



0.9967365967365968 3 1


In [ ]:
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

# **LOGISTIC REGRESSION**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}


imputer = KNNImputer(n_neighbors=1)
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
model = LogisticRegression(max_iter=1000000,solver="saga")
grid = GridSearchCV(model, grid, cv=10, scoring='accuracy', return_train_score=False)
grid.fit(X_train_imputed, y_train)
y_predicted = grid.predict(X_test_imputed)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)
            



# **Naive bayes**

In [18]:
# without scaling gaussian
from sklearn.naive_bayes import GaussianNB
imputer = KNNImputer(n_neighbors=1)
model3 = GaussianNB()
model3.fit(X_train_imputed, y_train)
y_predicted = model3.predict(X_test_imputed)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)


0.9687645687645687


In [20]:
#with min-max scaling gaussian
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train_imputed)
scaled_X_test = scaler.transform(X_test_imputed)
model3 = GaussianNB()
model3.fit(scaled_X_train, y_train)
y_predicted = model3.predict(scaled_X_test)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)


0.9748251748251748


In [21]:
#with min-max scaling gaussian and power transformer
from sklearn.preprocessing import PowerTransformer, StandardScaler
transformer = PowerTransformer()
X_train_transformed = transformer.fit_transform(X_train_imputed)
X_test_transformed = transformer.transform(X_test_imputed)
scaled_X_train = scaler.fit_transform(X_train_transformed)
scaled_X_test = scaler.transform(X_test_transformed)
model3 = GaussianNB()
model3.fit(scaled_X_train, y_train)
y_predicted = model3.predict(scaled_X_test)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)

0.9902097902097902


In [22]:
#without min-max scaling bernoulli
from sklearn.naive_bayes import BernoulliNB
model3 = BernoulliNB()
model3.fit(X_train_imputed, y_train)
y_predicted = model3.predict(X_test_imputed)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)



0.7752913752913753


In [23]:
#with min-max scaling bernoulli
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train_imputed)
scaled_X_test = scaler.transform(X_test_imputed)
model3 = BernoulliNB()
model3.fit(scaled_X_train, y_train)
y_predicted = model3.predict(scaled_X_test)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)

0.7757575757575758


In [24]:
#with min-max scaling bernoulli and power transformer
from sklearn.preprocessing import PowerTransformer, StandardScaler
transformer = PowerTransformer()
X_train_transformed = transformer.fit_transform(X_train_imputed)
X_test_transformed = transformer.transform(X_test_imputed)
scaled_X_train = scaler.fit_transform(X_train_transformed)
scaled_X_test = scaler.transform(X_test_transformed)
model3 = BernoulliNB()
model3.fit(scaled_X_train, y_train)
y_predicted = model3.predict(scaled_X_test)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)

0.7757575757575758


In [25]:
#without min-max scaling multinomial
from sklearn.naive_bayes import MultinomialNB
model3 = MultinomialNB()
model3.fit(X_train_imputed, y_train)
y_predicted = model3.predict(X_test_imputed)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)


0.9221445221445221


In [26]:
#with min-max scaling multinomial
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_X_train = scaler.fit_transform(X_train_imputed)
scaled_X_test = scaler.transform(X_test_imputed)
model3 = MultinomialNB()
model3.fit(scaled_X_train, y_train)
y_predicted = model3.predict(scaled_X_test)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)

0.7575757575757576


In [27]:
#with min-max scaling multi and power transformer
from sklearn.preprocessing import PowerTransformer, StandardScaler
transformer = PowerTransformer()
X_train_transformed = transformer.fit_transform(X_train_imputed)
X_test_transformed = transformer.transform(X_test_imputed)
scaled_X_train = scaler.fit_transform(X_train_transformed)
scaled_X_test = scaler.transform(X_test_transformed)
model3 = MultinomialNB()
model3.fit(scaled_X_train, y_train)
y_predicted = model3.predict(scaled_X_test)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)

0.7571095571095571


# **SVM**

In [28]:
grid={"C":[0.1,0.5,1.0], "gamma":[0.0,0.5,1.0],"kernel":["linear","rbf","poly","sigmoid"]}
from sklearn.svm import SVC
svm = SVC(kernel="rbf", gamma=0.5, C=1.0)
grid = GridSearchCV(svm, grid, cv=10, scoring='accuracy', return_train_score=False)
transformer = PowerTransformer()
X_train_transformed = transformer.fit_transform(X_train_imputed)
X_test_transformed = transformer.transform(X_test_imputed)
scaled_X_train = scaler.fit_transform(X_train_transformed)
scaled_X_test = scaler.transform(X_test_transformed)

grid.fit(scaled_X_train, y_train)
y_predicted = grid.predict(scaled_X_test)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

0.9967365967365968
0.9960924812822947
{'C': 1.0, 'gamma': 1.0, 'kernel': 'rbf'}
SVC(gamma=1.0)


# **DECISION TREE**

In [ ]:

param_grid = {'max_features': ['auto', 'sqrt', 'log2'],
              'ccp_alpha': [0.1, .01, .001],
              'max_depth' : [5, 6, 7, 8, 9],
              'criterion' :['gini', 'entropy']
             }
tree_clas = DecisionTreeClassifier(random_state=1024)
grid_search = GridSearchCV(estimator=tree_clas, param_grid=param_grid, cv=5, verbose=True)
grid.fit(scaled_X_train, y_train)
y_predicted = grid.predict(scaled_X_test)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)

In [30]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier                          
clf = BaggingClassifier(estimator=SVC(),n_estimators=10, random_state=0).fit(scaled_X_train, y_train)
y_predicted = clf.predict(scaled_X_test)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)

0.9972027972027973


In [44]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier                          

param_grid = {
    'n_estimators': [10, 50, 100],  
    'max_samples': [0.5, 1.0],  
    'max_features': [0.5,1],  
}

grid_search = GridSearchCV(BaggingClassifier(estimator=SVC()), param_grid, cv=5, verbose=1)
grid_search.fit(scaled_X_train, y_train)
y_predicted = grid.predict(scaled_X_test)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
0.9967365967365968
0.9960924812822947
{'C': 1.0, 'gamma': 1.0, 'kernel': 'rbf'}
SVC(gamma=1.0)


In [38]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier                          
clf = BaggingClassifier(estimator=KNeighborsClassifier(),n_estimators=10, random_state=0).fit(scaled_X_train, y_train)
y_predicted = clf.predict(scaled_X_test)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)

0.9948717948717949


In [40]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.ensemble import BaggingClassifier                          
clf = BaggingClassifier(estimator=RandomForestClassifier(),n_estimators=10, random_state=0).fit(scaled_X_train, y_train)
y_predicted = clf.predict(scaled_X_test)
from sklearn.metrics import accuracy_score

accuracy=accuracy_score(y_test, y_predicted);
print(accuracy)

0.9948717948717949
